In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 
import random as rand
import os

In [2]:
train_data=pd.read_csv("titanic_train-Copy1.csv")
test_data=pd.read_csv("titanic_test-Copy1.csv")

In [3]:
train_data.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [7]:
display(train_data.isnull().sum().sort_values(ascending=False))
display(test_data.isnull().sum().sort_values(ascending=False))

Cabin          687
Age            177
Embarked         2
Fare             0
Ticket           0
Parch            0
SibSp            0
Sex              0
Name             0
Pclass           0
Survived         0
PassengerId      0
dtype: int64

Cabin          327
Age             86
Fare             1
Embarked         0
Ticket           0
Parch            0
SibSp            0
Sex              0
Name             0
Pclass           0
PassengerId      0
dtype: int64

In [8]:
train_data['Cabin_new']=train_data['Cabin'].str[0]

In [9]:
train_data.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cabin_new
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,NaN
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,C


In [10]:
train_data['Age'].fillna(train_data['Age'].mean(),inplace=True)
train_data['Fare'].fillna(train_data['Fare'].median(),inplace=True)
train_data['Embarked'].fillna('S',inplace=True)
train_data['Cabin_new'].fillna('X',inplace=True)

In [11]:
train_data = train_data[train_data["Cabin_new"] != "T"]

In [12]:
train_data.head(8)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cabin_new
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S,X
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S,X
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S,C
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S,X
5,6,0,3,"Moran, Mr. James",male,29.699118,0,0,330877,8.4583,NaN,Q,X
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.000000,0,0,17463,51.8625,E46,S,E
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.000000,3,1,349909,21.0750,NaN,S,X


In [13]:
#Create new Cabin variable
test_data['Cabin_new'] = test_data['Cabin'].str[0]

#Fare
test_data['Fare'].fillna(train_data['Fare'].mean(), inplace=True)

#Age
test_data['Age'].fillna(train_data['Age'].mean(), inplace=True)
    
#Cabin
test_data['Cabin_new'].fillna('X', inplace=True)

In [14]:
test_data.head(6)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cabin_new
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,X
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,X
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,X
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,X
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,X
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S,X


In [15]:
y = train_data["Survived"]
features = ["Pclass", "Sex", "SibSp", "Parch", "Embarked", "Cabin_new" "Fare", "Age"]

X = pd.get_dummies(train_data[features[0:5]])
X["Fare"] = train_data["Fare"]
X["Age"] = train_data["Age"]

X_test = pd.get_dummies(test_data[features[0:5]])
X_test["Fare"] = test_data["Fare"]
X_test["Age"] = test_data["Age"]

In [16]:
model = RandomForestClassifier(random_state=126)

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('model', model)])

param_grid = {
    'model__n_estimators': [20, 30, 40],
    'model__max_depth': [7, 10, 13]
}

search = GridSearchCV(my_pipeline, param_grid, n_jobs=-1, verbose=10, cv=5)
search.fit(X, y)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   51.8s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   52.7s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   53.7s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   54.4s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   55.6s
[Parallel(n_jobs=-1)]: Done  43 out of  45 | elapsed:   56.5s remaining:    2.5s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   56.8s finished


Best parameter (CV score=0.834):
{'model__max_depth': 10, 'model__n_estimators': 30}


In [17]:
predictions = search.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
